In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from sklearn import feature_extraction, linear_model, model_selection, metrics
from sklearn import ensemble
from scipy import sparse
import sys
sys.path.append("..")
from src.make_model_lstm import hate_speech_model
from src.lstm_cleaner import lstm_cleaner
import tensorflow as tf
from tensorflow.core.protobuf import rewriter_config_pb2
from tensorflow.keras.backend import set_session

In [3]:
%load_ext autoreload
%autoreload 2

This is an example of running our data through our LSTM pipeline and then pushing that through our LSTM.

In [4]:
#tensorflow plays weirdly in notebook sometimes, this block takes care of that
tf.keras.backend.clear_session()  # For easy reset of notebook state.

config_proto = tf.ConfigProto()
off = rewriter_config_pb2.RewriterConfig.OFF
config_proto.graph_options.rewrite_options.arithmetic_optimization = off
session = tf.Session(config=config_proto)
set_session(session)

In [5]:
df = pd.read_csv('../data/labeled_data.csv')

In [6]:
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [7]:
c = lstm_cleaner()
sequences = c.fit_transform(df['tweet'])

In [8]:
y = df['class']

In [9]:
y_cat = np.zeros((len(y), 3))
for i, l in enumerate(y):
    y_cat[i, l] = 1

In [10]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(sequences, y_cat, test_size = .1)

In [11]:
#takes a while to run
model = hate_speech_model()
model.build_model()
model.fit(X_train, y_train, epochs = 100, class_weight = {0: 1, 1: .05, 2: .15})
preds = model.predict(X_test)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
>> Compiled...
Train on 21188 samples, validate on 1116 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
21188/21188 [==============================] - 59s 3ms/sample - loss: 0.1324 - val_loss: 0.1281
Epoch 2/100
21188/21188 [==============================] - 57s 3ms/sample - loss: 0.1310 - val_loss: 0.1267
Epoch 3/100
21188/21188 [==============================] - 57s 3ms/sample - loss: 0.1292 - val_loss: 0.1247
Epoch 4/100
21188/21188 [==============================] - 57s 3ms/sample - loss: 0.1269 - val_loss: 0.1228
Epoch 5/100
21188/21188 [==============================] - 57s 3ms/sample - loss: 0.1252 - val_loss: 0.1221
Epoch 6/100
21188/21188 [==============================] - 57s 3ms

In [12]:
print(metrics.roc_auc_score(y_test, preds))

0.8671014677571813


In [13]:
y_true = np.argmax(y_test, axis = 1)
y_pred = np.argmax(preds, axis = 1) 

In [14]:
print(metrics.classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.17      0.65      0.27       166
           1       0.94      0.81      0.87      1883
           2       0.86      0.46      0.60       430

    accuracy                           0.74      2479
   macro avg       0.65      0.64      0.58      2479
weighted avg       0.87      0.74      0.78      2479



In [15]:
print(metrics.confusion_matrix(y_true, y_pred))

[[ 108   52    6]
 [ 338 1518   27]
 [ 180   53  197]]
